In [1]:
import torch
from torch.nn import functional as F
from torch import nn
import math
import random
import numpy as np
import requests

In [2]:
block_size = 8
batch_size = 32
epochs = 4000
eval_iters = 300

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    torch.backends.cudnn.benchmark = True

learning_rate = 5e-3

In [3]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        self.embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, ix, targets=None): # ix is the index represented by a B,T,C tensor with character tokens

        logits = self.embedding_table(ix)

        if targets == None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(input=logits, target=targets)

        return logits, loss
    
    def generate(self, n_chars, ix):
        
        for _ in range(n_chars):

            logits, loss = self(ix) # B, T, C
            logits = logits[:,-1,:] # B, C -- we need to reshape to calculate probabilities
            probs = F.softmax(logits, dim=-1) # B, C
            next_ix = torch.multinomial(input=probs, num_samples=1)
            ix = torch.cat((ix, next_ix), dim=1)

        return ix

In [4]:
data = open('text.txt').read()
vocab = list(sorted(set(data)))
vocab_size = len(vocab)

print(f'Vocab: {vocab[:10]}.')
print(f'Vocab size: {vocab_size}')

stoi = {c:i for i, c in enumerate(vocab)}
itos = {i:c for i, c in enumerate(vocab)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda i: ''.join([itos[i] for i in i])

print(decode(encode(data[1100:1150])))
data = torch.tensor(encode(data))

Vocab: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3'].
Vocab size: 65
very dog to the commonalty.

Second Citizen:
Consi


In [5]:
n_tr = int(len(data) * 0.9)
n_val = len(data) - n_tr

train = data[:n_tr]
val = data[n_tr+1:]

print(f'Training samples: {train.shape[0]}')
print(f'Validation samples: {val.shape[0]}')

def make_batches(split):

    data = train if split == 'train' else val
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    
    return x, y

Training samples: 1003854
Validation samples: 111539


In [6]:
# read the data
# create vocab
# do vocab size
# create training and validation splits
# create data loader
# create the bi-gram lookup table
# train and predict with the Bigram. 
# print validation during training

Cross Entropy - Torch vs. Manual calculation

In [7]:
# Learn how cross entropy works.
# cross_entropy method vs manual way

Xb, Yb = make_batches('train')

emb_table = nn.Embedding(vocab_size, vocab_size).to(device) # (65, 65)
logits = emb_table(Xb) # (32, 8, 65)
print(f'Input with embeddings: {logits.shape}')
B, T, C = logits.shape
logits = logits.view(B*T, C)
targets = Yb.view(B*T)
loss = F.cross_entropy(logits, targets)

logits_i = logits[0]
target_i = targets[0]

loss_i = F.cross_entropy(logits_i.unsqueeze(0), target_i.unsqueeze(0))
# ------------------------------------------------------------#
probabilities = torch.softmax(logits_i, dim=0)
nll = -torch.log(probabilities[target_i])
print(f'Are both calculations identical: {loss_i.item():.6f}, {nll.item():.6f}')

Input with embeddings: torch.Size([32, 8, 65])
Are both calculations identical: 4.177806, 4.177806


In [13]:
# Model definition
m = BigramLanguageModel(vocab_size=vocab_size).to(device)
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
n_params = sum(p.nelement() for p in m.parameters())
print(f'Total parameteres: {n_params}')

Total parameteres: 4225


In [9]:
@torch.no_grad()
def estimate_loss(m):
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = make_batches(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [10]:
# Training
for epoch in range(epochs):

    Xb, Yb = make_batches('train')
    logits, loss = m(Xb, Yb) # B, C

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if epoch % 500 == 499:
        l = estimate_loss(m)
        print(f"Iteration {epoch}. Training Loss: {l['train']:.3f}. Evaluation Loss: {l['val']:.3f}")

Iteration 499. Training Loss: 2.944. Evaluation Loss: 2.964
Iteration 999. Training Loss: 2.589. Evaluation Loss: 2.600
Iteration 1499. Training Loss: 2.514. Evaluation Loss: 2.533
Iteration 1999. Training Loss: 2.491. Evaluation Loss: 2.497
Iteration 2499. Training Loss: 2.478. Evaluation Loss: 2.489
Iteration 2999. Training Loss: 2.475. Evaluation Loss: 2.489
Iteration 3499. Training Loss: 2.469. Evaluation Loss: 2.491
Iteration 3999. Training Loss: 2.463. Evaluation Loss: 2.486


In [11]:
context = torch.zeros(1,1).int().to(device)
predictions = m.generate(n_chars=500, ix=context)
print(decode(predictions[0].tolist()))


LEx

TCKICant,
Thavet ho If t InavareUKI t, Gore forler s m!
3 gs an, t s't.
Angonchem cin s the gowhe ht whould f lo d, t gakur pldousathewir han
MESen , ayowid:
Annoucheathant
M:
POF IORDUTElNRoug, fl Ang pon is anlave ns tsthere sort vere mane theanaiuthe ad the pe weckithe g?
BENe.
Jor no, thort, malise Homyo II gheno as
DUETHe,
AUK:
SS:
T: ile mspo Von it ugh ivis, t f the s stre wree trton thor averldomedis cin.
An. n,
We d men ro finy cke ySove theity itheve h de seri-stheshathes hell kea
